In [1]:
! pip install confluent-kafka avro-python3 requests protobuf

  Preparing metadata (setup.py) ... done
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=43991 sha256=04847fe19397b785da994b0788684b6ef5ec012e7207347df1b9bce44e1565c7
  Stored in directory: /home/jovyan/.cache/pip/wheels/31/be/50/1145d9510eb4440893fc0ec676ef9464a05e0f7492a76fbb2c
Successfully built avro-python3


In [2]:
! protoc --python_out=. ./search_result.proto

In [3]:
import sys
import os

# Get the directory where search_request_pb2.py is located
proto_dir = os.getcwd()  # Or specify the absolute path if needed
sys.path.append(proto_dir)
import search_result_pb2
from search_result_pb2 import SearchRequest

import random
import time
import requests
from confluent_kafka import Producer


# Kafka and Schema Registry Config
KAFKA_BROKER = "redpanda:9092"
SCHEMA_REGISTRY_URL = "http://redpanda:8081"
TOPIC_NAME = "search_requests"

# Configure Kafka Producer
producer = Producer({
    "bootstrap.servers": KAFKA_BROKER,
    "client.id": "protobuf-producer"
})

# Register Schema in Redpanda's Schema Registry
schema_subject = f"{TOPIC_NAME}-value"
schema_payload = {
    "schemaType": "PROTOBUF",
    "schema": """
        syntax = "proto3";
        message SearchRequest {
            string query = 1;
            int32 page_number = 2;
            int32 results_per_page = 3;
        }
    """
}

# Check if schema already exists
response = requests.get(f"{SCHEMA_REGISTRY_URL}/subjects/{schema_subject}/versions")
if response.status_code == 404:
    # Register schema if it doesn't exist
    reg_response = requests.post(
        f"{SCHEMA_REGISTRY_URL}/subjects/{schema_subject}/versions",
        json=schema_payload
    )
    print("Schema registered:", reg_response.json())
else:
    print("Schema already registered.")

# Function to generate random SearchRequest events
def generate_random_event():
    search_request = SearchRequest(
        query=random.choice(["kafka", "redpanda", "protobuf", "streaming"]),
        page_number=random.randint(1, 10),
        results_per_page=random.randint(10, 50),
    )
    return search_request.SerializeToString()

# Kafka message delivery report callback
def delivery_report(err, msg):
    if err:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

# Publish messages to Kafka
for _ in range(10):
    event_data = generate_random_event()
    producer.produce(TOPIC_NAME, value=event_data, callback=delivery_report)
    producer.flush()
    time.sleep(1)  # Simulate event streaming

print("✅ Events published successfully!")


Schema registered: {'id': 1}
Message delivered to search_requests [0]
Message delivered to search_requests [0]
Message delivered to search_requests [0]
Message delivered to search_requests [0]
Message delivered to search_requests [0]
Message delivered to search_requests [0]
Message delivered to search_requests [0]
Message delivered to search_requests [0]
Message delivered to search_requests [0]
Message delivered to search_requests [0]
✅ Events published successfully!
